In [1]:
from pyspark import SparkConf,SparkContext

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("spark_try")
sc=SparkContext(conf=conf)

23/12/30 14:59:50 WARN Utils: Your hostname, hewens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.2.11 instead (on interface en0)
23/12/30 14:59:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/30 14:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/12/31 01:29:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 173646 ms exceeds timeout 120000 ms
23/12/31 01:29:01 WARN SparkContext: Killing executors is not supported by current scheduler.
23/12/31 01:29:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

In [3]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9],3)
rdd.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [8]:
data = [12,3,4,5,6,7,8]
new_rdd = sc.parallelize(data)
new_rdd.getNumPartitions()

8

In [9]:
file_rdd = sc.textFile('./words.txt')
file_rdd.getNumPartitions()

2

In [11]:
file_rdd_flat = file_rdd.flatMap(lambda x:x.split())
file_rdd_flat.collect()

['spark',
 'spark',
 'sql',
 'sql',
 'sql',
 'python',
 'python',
 'github',
 'java',
 'java',
 'cpp',
 'cpp']

In [14]:
words_count = file_rdd_flat.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
words_count.collect()

[('python', 2),
 ('java', 2),
 ('cpp', 2),
 ('spark', 2),
 ('sql', 3),
 ('github', 1)]

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("spark_sql_test") \
    .getOrCreate() 

23/12/31 11:05:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
sc = spark.sparkContext

In [8]:
words = spark.read.csv('/Users/hewenpang/Documents/tf_explore/Pyspark/words.txt',sep=' ')
words.show()

+------+------+
|   _c0|   _c1|
+------+------+
| spark| spark|
|   sql|   sql|
|python|python|
|github|  java|
|  java|   cpp|
+------+------+



In [9]:
words_2 = spark.read.format('text').load('./words.txt')
words_2.printSchema()
words_2.show()

root
 |-- value: string (nullable = true)

+-------------+
|        value|
+-------------+
|  spark spark|
|  sql sql sql|
|python python|
|  github java|
| java cpp cpp|
+-------------+



In [13]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [15]:
schema = StructType().add('user_id',StringType(),nullable=True).\
add('movie_id',StringType(),nullable=True).\
add('rank',IntegerType(),nullable=True).\
add('ts',StringType(),nullable=True)
data = spark.read.format('csv').option('sep','\t').option('header',False).\
    option('encoding','utf-8').schema(schema=schema).load('/Users/hewenpang/Documents/tf_explore/Pyspark/u.data.txt')
data.show()

+-------+--------+----+---------+
|user_id|movie_id|rank|       ts|
+-------+--------+----+---------+
|    196|     242|   3|881250949|
|    186|     302|   3|891717742|
|     22|     377|   1|878887116|
|    244|      51|   2|880606923|
|    166|     346|   1|886397596|
|    298|     474|   4|884182806|
|    115|     265|   2|881171488|
|    253|     465|   5|891628467|
|    305|     451|   3|886324817|
|      6|      86|   3|883603013|
|     62|     257|   2|879372434|
|    286|    1014|   5|879781125|
|    200|     222|   5|876042340|
|    210|      40|   3|891035994|
|    224|      29|   3|888104457|
|    303|     785|   3|879485318|
|    122|     387|   5|879270459|
|    194|     274|   2|879539794|
|    291|    1042|   4|874834944|
|    234|    1184|   2|892079237|
+-------+--------+----+---------+
only showing top 20 rows



In [18]:
#用户平均分
data.groupBy('user_id').avg('rank').show()

+-------+------------------+
|user_id|         avg(rank)|
+-------+------------------+
|    296|4.1768707482993195|
|    467|3.6818181818181817|
|    691|           4.21875|
|    675|3.7058823529411766|
|    829|          3.546875|
|    125|3.4395604395604398|
|    451|2.7346938775510203|
|    800|              3.75|
|    853| 2.975609756097561|
|    666|3.6653061224489796|
|    870| 3.453531598513011|
|    919|3.4700460829493087|
|    926|               3.3|
|      7| 3.965260545905707|
|    124|               3.5|
|     51|3.5652173913043477|
|    447| 3.597122302158273|
|    591|3.6547619047619047|
|    307| 3.794642857142857|
|    475|               3.6|
+-------+------------------+
only showing top 20 rows



In [35]:
data.where(data['rank']>data.select(F.avg(data['rank'])).first()['avg(rank)']).count()

55375

24/01/01 04:14:16 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 925391 ms exceeds timeout 120000 ms
24/01/01 04:14:16 WARN SparkContext: Killing executors is not supported by current scheduler.
24/01/01 04:14:26 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B